# [실습2] LLM와 Langchain으로 데이터 분석 그래프 그리기

## 실습 목표
---
Langchain과 LLM을 결합해서 주어진 데이터를 분석하여 그래프로 보여주는 파이썬 코드를 생성하고, 이를 Langchain을 활용해 실행하는 법을 학습합니다.

## 실습 목차
---

1. **데이터 그래프 생성 체인 구성:** 데이터를 분석하여 대응하는 그래프를 생성하는 코드를 생성하고 실행하는 체인을 구성합니다.

## 실습 개요
---
LangChain을 활용해서 자연어로 데이터를 분석할 수 있는 챗봇을 구현하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [ ]:
import contextlib
import io
import os

import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_experimental.tools.python.tool import PythonAstREPLTool

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [ ]:
!ollama pull mistral:7b

## 1. 데이터 특징 추출
- 주어진 데이터가 어떻게 구성되어 있는지 추출하고 이를 시스템 프롬프트에 적용합니다.
- 이번 실습에 사용할 데이터는 2일차 실습 중 **Langchain을 이용한 머신러닝 기반 데이터 분석 실무 프로젝트** 에 사용하는 데이터입니다.

먼저, mistral:7b 모델을 사용하는 ChatOllama 객체와 OllamaEmbeddings 객체를 생성합니다.

In [ ]:
llm = ChatOllama(model="mistral:7b")
embeddings = OllamaEmbeddings(model="mistral:7b")

다음으로, 데이터를 불러오고, 데이터의 컬럼명을 변수에 저장합니다.

대부분의 경우 컬럼명에서 데이터의 특성을 파악할 수 있기 때문에, LLM이 사용자의 질문에 맞는 데이터를 어떤 코드를 통해 추출할지 추론하는 좋은 단서가 됩니다.

In [ ]:
# 데이터를 불러오고, 이름과 컬럼명을 저장합니다.
data_dir = './data'
df_inkjet = pd.read_csv(os.path.join(data_dir, 'InkjetDB_preprocessing.csv'), index_col=0)

# 데이터를 저장한 변수명을 LLM에 제공하여 이 변수를 활용하는 코드를 작성하게 할 수 있습니다.
df_name = "df_inkjet"
df_columns = ", ".join(df_inkjet.columns)

다음으로, 데이터의 컬럼명을 바탕으로 **그래프를** 그리는 코드를 생성하는 프롬프트를 작성합니다.

In [ ]:
system_message = "당신은 주어진 데이터를 분석하는 데이터 분석가입니다.\n"
system_message += f"주어진 DataFrame에서 데이터를 추출하여 사용자의 질문에 답할 수 있는 그래프를 그리는 코드를 작성하세요. "
system_message += f"{df_name} DataFrame에 액세스할 수 있습니다.\n"
system_message += f"`{df_name}` DataFrame에는 다음과 같은 열이 있습니다: {df_columns}\n"
system_message += "데이터는 이미 로드되어 있으므로 데이터 로드 코드를 생략해야 합니다."

message_with_data_info = [
    ("system", system_message),
    ("human", "{question}"),
]

시스템 프롬프트를 확인해 봅시다.

In [ ]:
print(system_message)

이전 실습에서 구성한 코드를 생성하는 체인을 구성하고 실행해 봅시다.

In [ ]:
# LLM이 생성한 코드를 파싱하는 함수를 정의합니다.
def python_code_parser(input: str) -> str:
    # LLM은 대부분 ``` 블럭 안에 코드를 출력합니다. 이를 활용합니다.
    # ```python (코드) ```, 혹은 ``` (코드) ``` 형태로 출력됩니다. 두 경우 모두에 대응하도록 코드를 작성합니다.
    processed_input = input.replace("```python", "```").strip()
    parsed_input_list = processed_input.split("```")

    # 만약 ``` 블럭이 없다면, 입력 텍스트 전체가 코드라고 간주합니다.
    # 아닐 경우 이어지는 코드 실행 과정에서 예외 처리를 통해 오류를 확인할 수 있습니다.
    if len(parsed_input_list) == 1:
        return processed_input

    # 코드 부분만 추출합니다. 
    # LLM은 여러 코드 블럭에 걸쳐 필요한 코드를 출력할 수 있으므로, 코드가 있는 홀수 번째 텍스트를 모두 저장합니다.
    parsed_code_list = []
    for i in range(1, len(parsed_input_list), 2):
        parsed_code_list.append(parsed_input_list[i])
    
    # 코드 부분을 하나로 합칩니다.
    return "\n".join(parsed_code_list)

In [ ]:
# 생성한 코드를 실행하는 함수를 정의합니다.
def run_code(input_code: str):
    # 코드가 출력한 값을 캡쳐하기 위한 StringIO 객체를 생성합니다.
    output = io.StringIO()
    try:
        # Redirect stdout to the StringIO object
        with contextlib.redirect_stdout(output):
            # Python 3.10 버전이므로, 키워드 인자를 사용할 수 없습니다.
            # 코드가 실행하면서 출력한 모든 결과를 캡쳐합니다.
            exec(input_code, {"df_inkjet": df_inkjet})
    except Exception as e:
        # 에러가 발생할 경우, 이를 StringIO 객체에 저장합니다.
        print(f"Error: {e}", file=output)
    # StringIO 객체에 저장된 값을 반환합니다.
    return output.getvalue()

In [ ]:
prompt_with_data_info = ChatPromptTemplate.from_messages(message_with_data_info)

# 체인을 구성합니다.
code_execute_chain = (
    {"question": RunnablePassthrough()}
    | prompt_with_data_info
    | llm
    | StrOutputParser()
    | python_code_parser
    | run_code
)

구성한 체인을 실행해 봅시다.

In [ ]:
print(code_execute_chain.invoke("데이터 분포를 그려줘"))

대부분의 경우, 질문에 맞는 그래프가 나타납니다.

이는 체인에 포함된 `run_code` 함수에서 그래프를 그리는 코드 (대부분 `plt.plot()`)를 실행하면서 그래프가 그대로 출력되는 것입니다.

또한, '저장해줘' 같은 표현을 프롬프트에 추가할 경우, 그린 그래프를 저장하는 코드까지 실행하면서 질문에 대응하는 그래프를 파일로 저장합니다


In [ ]:
print(code_execute_chain.invoke("데이터 분포를 그리고 저장해줘."))

`Ctrl/Cmd+S`를 눌러 저장한 후, 왼쪽 위에 있는 Jupyter 로고를 눌러 초기 화면으로 돌아가면 그래프 파일이 저장되어 있을 것입니다.

### 추가 실습
- 간혹, 그래프 대신 흰색 화면만 저장되는 경우가 있습니다. 이는 plt.savefig() 함수를 plt.show() 보다 나중에 호출해서 생기는 문제입니다.
- LLM이 생성한 코드를 확인하고, plt.savefig() 함수와 plt.show() 함수가 둘 다 있다면 둘의 위치를 바꾸는 함수를 구현해서 체인에 추가해보세요.

In [ ]:
def verify_and_fix_plot_code(input_code: str) -> str:
    # LLM이 생성한 코드를 확인하고, plt.savefig() 함수가 plt.show() 함수 뒤에 있다면 둘의 순서를 바꿔서 반환합니다.
    # plt.show()를 plt.savefig('plot.png') 로 바꿔도 됩니다.
    fixed_code = input_code
    ##############################
    # 여기에 코드를 작성하세요.
    ##############################
    return fixed_code

######################
# 코드를 구현했다면, 이를 추가한 체인을 구성하고, 챗봇으로 만들어서 테스트합니다


######################

In [ ]:
#예시 코드

import re

def verify_and_fix_plot_code(input_code: str) -> str:
    # LLM이 생성한 코드를 확인하고, plt.savefig() 함수가 plt.show() 함수 뒤에 있다면 둘의 순서를 바꿔서 반환합니다.
    # plt.show()를 plt.savefig('plot.png') 로 바꿔도 됩니다.
    fixed_code = input_code
    ##############################
    savefig_pattern = r"(plt\.savefig\([^\)]*\))"
    show_pattern = r"(plt\.show\(\))"

    # 두 명령의 위치를 검색
    savefig_match = re.search(savefig_pattern, fixed_code)
    show_match = re.search(show_pattern, fixed_code)

    # plt.savefig()가 plt.show() 위에 있을 경우 위치를 바꿈
    if savefig_match and show_match and savefig_match.start() < show_match.start():
        # 코드에서 명령어 추출
        savefig_code = savefig_match.group(0)
        show_code = show_match.group(0)

        # plt.savefig()와 plt.show()의 순서 교환
        fixed_code = (
            fixed_code[:savefig_match.start()] + 
            fixed_code[savefig_match.end():show_match.start()] +
            savefig_code + "\n" + 
            show_code + 
            fixed_code[show_match.end():]
        )
    ##############################
    return fixed_code

######################
# 코드를 구현했다면, 이를 추가한 체인을 구성하고, 챗봇으로 만들어서 테스트합니다
change_code_execute_chain = (
    {"question": RunnablePassthrough()}
    | prompt_with_data_info
    | llm
    | StrOutputParser()
    | python_code_parser
    | verify_and_fix_plot_code
    | run_code
)

######################

print(change_code_execute_chain.invoke("데이터 분포를 그리고 저장해줘."))